Given the OHLCV data from binance, create all the needed features 

In [6]:
import ccxt
import pandas as pd
import polars as pl
import os

# GLOBAL VARIABLES
# change to rel. paths
TICKER_DATA_PATH = r"C:\Users\Damja\CODING_LOCAL\trading\data\ticker_specific_data_BINANCE"

In [ ]:
### TO-DO's
- create checks for correct dates
- test data quality
- create features

In [7]:
pairs = pd.read_csv("pairs.csv")
NUM_PAIRS_TO_LOAD = 100
pairs = pairs.iloc[:NUM_PAIRS_TO_LOAD, 0].values
pairs = [pair.replace("USD", "USDT") for pair in pairs]

'scan' uses lazy evaluation --> optimized

In [2]:
pair = pairs[2]

NameError: name 'pairs' is not defined

In [33]:
df = pl.read_parquet(f'{TICKER_DATA_PATH}/{pair.replace("/", "")}.parquet').to_pandas()

In [1]:
#df.collect_schema().names()

#### Feature Creation
- 30 mins momentum
- change in volume
- current vol / 24h avg volume (check for weekends?) --> maybe a flag; is weekend or is change to weekend

In [ ]:
#df.with_columns['usd_volume_30m_delta'] = df['usd_volume'].diff()

df['usd_volume_30m_delta'] = df['usd_volume'].diff()
df['usd_volume_60m_delta'] = df['usd_volume'].diff(2)
df['usd_volume_3hm_delta'] = df['usd_volume'].diff(6)

df['return_30m'] = df['close'].diff()/df['close']*100
df['return_60m'] = df['close'].diff(2)/df['close']*100
df['return_3h'] = df['close'].diff(6)/df['close']*100

Calculate the rolling 6h Low and High prices; It's difference is a good proxy of coin specific volatility within this timeframe

In [ ]:
df['6h_low'] = df['low'].rolling(12).min()
df['6h_high'] = df['high'].rolling(12).max()
df['6h_high_minus_low'] = df['6h_high'] - df['6h_low']

df['6h_close_volatility'] = df['close'].rolling(12).std()

df.columns

In [ ]:
df['Date'] > pd.to_datetime("2025-01-01")

### Training Step

What is the naive benchmark that we need to be better than when looking at just 1 pair?
- Better than long term buy and hold?
- better in terms of risk measures (max DD, sharpe,..)


One idea is not to always be invested but only in lucrative phases. This would lend to using leverage in these phases. Such a strategy would need to be evealuated based on drawdown events and hit ratio of the strategy in general. Either lower hit ratio with high payout potential or high hit ratio with higher leverage --> high conviction trades only.
The question is what models lend itself to such a strategy?